In [12]:
# import the library
from appJar import gui
import keras
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
#from load_data import *
import pandas as pd
import cv2
from random import randint
import math

#pre-process
#from preprocess.eyes_locator import eye_locator
#from preprocess.divider import img_rotator, row_cropper, digit_cropper
from os import listdir
from os.path import isfile, join

dataset_path = './Img2Classify/'
data = []

# Load Pretrained Model
model = load_model('./model_27.h5')

In [9]:
def eye_locator(source):

    new_image = cv2.imread(source)
    CYAN = (255, 255, 0)

    org_image = image = new_image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    ret, thresh = cv2.threshold(image, 115, 255, cv2.THRESH_BINARY)
    image = cv2.bitwise_not(thresh)


    _, contours, _ = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    eye_regions = []

    for contour in contours:

        (x, y, w, h) = cv2.boundingRect(contour)
        # Compare the width and height of the contour to detect eyes that
        # are conjoined into one chunk
        if w < 80 and h < 60 and w > 60 and h > 45 and (x < 300 or x > 1800):
            eye_regions.append((x, y, w, h))

    if len(eye_regions) == 30:

        # Sort the located eye images based on the x coordinate left column up-to-down, then right column up-to-down
        eye_regions = sorted(eye_regions, key=lambda x: (x[0],x[1]) )
        left_eye_regions = sorted(eye_regions[0:15], key=lambda x: (x[1]) )
        right_eye_regions = sorted(eye_regions[15:30], key=lambda x: (x[1]) )
        eye_regions = left_eye_regions + right_eye_regions

        # loop over the eyes
        for eye_bounding_box in eye_regions:
            # Grab the coordinates of the eye in the image
            x, y, w, h = eye_bounding_box

            # Draw border
            cv2.rectangle(org_image, (x - 2, y - 2), (x + w + 4, y + h + 4), CYAN, 5)

        # Save the modified image
        cv2.imwrite('cache/1.eye_detected/{image_name}-eyeLocated.jpg'.format(image_name=randint(1000000000, 9999999999)), org_image)
        
        # Return box details
        return cv2.imread(source), eye_regions

def img_rotator(image, eye_regions):

    x1, x2, y1, y2 = (eye_regions[7][0]), (eye_regions[22][0]), eye_regions[7][1], eye_regions[22][1]
    rows,cols = image.shape[:2]
    
    angle_rad = math.atan2(y2 - y1, x2 - x1)
    angle_deg = math.degrees(angle_rad)
    M = cv2.getRotationMatrix2D((cols/2,rows/2), angle_deg, 1)
    dst = cv2.warpAffine(image, M, (cols, rows))
    
    new_eye_regions = []
    
    for eye in eye_regions:
        new_x, new_y = point_around_point_rotator(eye[0], eye[1], cols/2, rows/2, (-1)*angle_rad)
        new_eye_regions.append((new_x, new_y, eye[2], eye[3]))

    cv2.imwrite('cache/2.rotated/{image_name}-itemDetected.jpg'.format(image_name=randint(1000000000, 9999999999)), dst)
    return dst, new_eye_regions
    
def point_around_point_rotator(x_point, y_point, x_origin, y_origin, angle_rad):
    qx = int(x_origin + math.cos(angle_rad) * (x_point - x_origin) - math.sin(angle_rad) * (y_point - y_origin))
    qy = int(y_origin + math.sin(angle_rad) * (x_point - x_origin) + math.cos(angle_rad) * (y_point - y_origin))
    return qx, qy

def row_cropper(img, left_eye, right_eye, row_id):
    starting_row = int(left_eye[1]-55)
    ending_row = int(left_eye[1]+90)
    starting_col = int(left_eye[0])
    ending_col = int(right_eye[0]+right_eye[2])
    
    cropped = img[starting_row:ending_row, starting_col:ending_col]

    #reset eye's coordinates
    right_eye_x = right_eye[0]-left_eye[0]
    right_eye_y = 55
    left_eye_x = 0
    left_eye_y = 55
    
    new_left_eye = (left_eye_x, left_eye_y, left_eye[2], left_eye[3])
    new_right_eye = (right_eye_x, right_eye_y, right_eye[2], right_eye[3])
    
    #cv2.circle(cropped,(left_eye_x, left_eye_y), 5, (0,0,255), -1)
    cv2.imwrite('cache/3.row_cropped/{0}-rowCropped.jpg'.format(randint(1000000000, 9999999999)), cropped)
    return cropped, new_left_eye, new_right_eye

def digit_cropper(cropped, left_eye, right_eye):
    
    row = int(left_eye[1]+left_eye[3]/2)-6
    col_Xs = [893, 1003, 1113, 1263, 1370]
    digits = []

    VERTICAL_MARGIN = 65
    HORIZONTAL_MARGIN = 53

    for i in range(5):
        img = cropped[row-VERTICAL_MARGIN:row+VERTICAL_MARGIN, col_Xs[i]-HORIZONTAL_MARGIN:col_Xs[i]+HORIZONTAL_MARGIN]
        img = cv2.resize(img, dsize=(100, 100))
        cv2.imwrite('cache/4.digit_cropper/{image_name}-itemDetected.jpg'.format(image_name=randint(1000000000, 9999999999)), img)
        digits.append(img)
    return digits

def predict():
    files = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f)) and join(dataset_path, f).endswith('.jpg')]
    files = sorted(files)
    
    for f in files:
        image, eye_regions = eye_locator(dataset_path + f)
        new_image, new_eye_regions = img_rotator(image, eye_regions)
        data = []
        
        ###########
        #output_img = cv2.imread(dataset_path + f)
        padding_left = 200
        ###########

        for i in range(0,15):
            cropped, left_eye, right_eye = row_cropper(new_image, new_eye_regions[i], new_eye_regions[i+15], str(i+1))
            digits_img = digit_cropper(cropped, left_eye, right_eye)

            digits_classes = []
            score = ''

            for j in range(len(digits_img)):

                digits_classes.append(model.predict_classes(np.array([digits_img[j]])))

            for k in range(3):
                if digits_classes[k] == 10 or digits_classes[k] == 11:
                    continue
                else:
                    score = score + str(digits_classes[k][0])

            score = score + '.'

            for k in range(3, 5):
                if digits_classes[k] == 10 or digits_classes[k] == 11:
                    continue
                else:
                    score = score + str(digits_classes[k][0])

            if score.endswith('.'):
                score = score[:-1]
##############
            starting_point = (new_eye_regions[i][0]+round(new_eye_regions[i][2]/2), new_eye_regions[i][1]+round(new_eye_regions[i][3]/2))
            #cv2.circle(new_image,(starting_point[0]+padding_left, starting_point[1]), 10, (0,0,255), -1)
            cv2.putText(new_image, score, (starting_point[0]+padding_left, starting_point[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
##############
            data.append([i+1, float(score)])
        #for i in range(15):
        #    cv2.circle(new_image,(new_eye_regions[i][0]+round(new_eye_regions[i][2]/2), new_eye_regions[i][1]+round(new_eye_regions[i][3]/2)), 10, (0,0,255), -1)
        df = pd.DataFrame(data,columns=['row','score'])
        df.to_csv("./output/image_{0}.csv".format(f), sep='\t')
        cv2.imwrite("./output/image_{0}".format(f), new_image)

In [10]:
# handle button events
def press(button):
    if button == "Exit":
        app.stop()
    else:
        predict()
        app.addLabel("success", "OCR was finished successfully!")


# create a GUI variable called app
app = gui("BRPS OCR", "400x200")
app.setBg("white")
app.setFont(18)

# add & configure widgets - widgets get a name, to help referencing them later
app.addLabel("title", "click predict 2 predict!")
app.setLabelBg("title", "gray")
app.setLabelFg("title", "orange")

# link the buttons to the function called press
app.addButtons(["[Predict]", "Exit"], press)

# start the GUI
app.go()